# Introduction

This notebook documents the extract, transform, load process for the QuantumTalent AI market analytics project.
The objective is to prepare a clean and standardized dataset for HR analytics by implementing a medallion
architecture composed of Bronze, silver and gold layers

What will this notebook include? 
- Database and schema initialization
- Bronze layer: raw data ingestion
- Silver layer: data cleaning and standardization
- The Gold layer: dimensional modeling, star schema, KPIs and measures 

# ETL Architecture Overview

This project follows a **medallion architecture** approach:

- Bronze layer: stores raw, unprocessed data as received from the source CSV file.
- Silver layer: contains cleaned, standardized, and business-ready data.
- Gold layer: analytical models and KPIs

goal of this separation: To improve data quality, traceability, and scalability of the analytics pipeline.


In [ ]:
# database intialization 

use master; 
create database ai_jobs; 

use ai_jobs;

#creation of bronze layer ( for row data )# 
create schema bronze; 
go 
#creation of silver layer ( for cleaned data ) # 
create schema silver; 
go 
# creation of gold layer ( for analysis ready data)# 
create schema gold;
go 

## Bronze Layer – Raw Data Ingestion

The Bronze layer stores raw AI job market data exactly as received from the source.
No transformations, validations, or business rules are applied at this stage.

The purpose of this layer is to:
- Preserve original data
- Enable traceability
- Act as a reliable ingestion source for downstream transformations


In [ ]:
use ai_jobs; 

#drop table if it existts
IF OBJECT_ID('bronze.ai_jobs', 'U') IS NOT NULL
BEGIN
    DROP TABLE bronze.ai_jobs;
END
GO 

CREATE TABLE bronze.ai_jobs (
    job_id                  VARCHAR(50),
    job_title               VARCHAR(255),
    salary_usd              VARCHAR(50),
    salary_currency         VARCHAR(10),
    experience_level        VARCHAR(50),
    employment_type         VARCHAR(50),
    company_location        VARCHAR(100),
    company_size            VARCHAR(50),
    employee_residence      VARCHAR(100),
    remote_ratio            VARCHAR(50),
    required_skills         VARCHAR(MAX),
    education_required      VARCHAR(100),
    years_experience        VARCHAR(50),
    industry                VARCHAR(100),
    posting_date            VARCHAR(50),
    application_deadline    VARCHAR(50),
    job_description_length  VARCHAR(50),
    benefits_score          VARCHAR(50),
    company_name            VARCHAR(255)
);
GO 


In [ ]:
# Load CSV into Bronze Layer 

TRUNCATE TABLE bronze.ai_jobs;
BULK INSERT bronze.ai_jobs
FROM 'C:\Users\user\Desktop\BIIIII\ai_job_dataset (2).csv'
WITH (
    FIRSTROW = 2,
    FIELDTERMINATOR = ',',
    ROWTERMINATOR = '\n',
    TABLOCK
);


## Bronze Layer Validation

After ingestion, basic validation checks are performed to ensure that:
- Data was loaded successfully
- Row counts are consistent
- Columns are populated correctly


In [ ]:
select * from bronze.ai_jobs; 

SELECT COUNT(*) AS total_rows
FROM bronze.ai_jobs;

SELECT TOP 10 *
FROM bronze.ai_jobs;


# Silver Layer : transformation of data

The **Silver layer** is the intermediate stage of the ETL pipeline. Its main purpose is to **clean, standardize, and structure raw data** from the Bronze layer, making it ready for analytics in the gold layer.


> **goal** Data is clean, validated, and structured, making it suitable for  building dimensions and fact tables in the Gold layer.


- ## Example of transformations carried

1. **Create Cleaned Tables**
   - Droped existing tables to avoid conflicts.
   - Defined tables with proper column types.
2. **Transform Raw Data**
   - Used `TRY_CAST` to safely convert numeric or date columns.
   - Applied `CASE` statements to categorize values.
   - Trim text fields to remove extra spaces.
   - Categorize numerical scores (e.g., benefits_score → Poor, Fair, Good, Excellent)
3. **Normalize skills column**
   - Split `skills` into a separate table for one-to-many relationships.
4. **Validate**
   - Check for nulls, duplicates, and malformed data before moving to the Gold layer.


In [ ]:
#DDL for silver table#

#Drop the silver table if it exists to avoind conflics
IF OBJECT_ID('silver.ai_jobs_clean', 'U') IS NOT NULL
BEGIN
    DROP TABLE silver.ai_jobs_clean;
END
GO 

#creation of the silver table
CREATE TABLE silver.ai_jobs_clean (
    job_id VARCHAR(50) PRIMARY KEY,
    job_title VARCHAR(255) NOT NULL,
    salary_usd DECIMAL(15,2),
    experience_level VARCHAR(50),
    employment_type VARCHAR(50),
    company_location VARCHAR(100),
    company_size VARCHAR(50),
    employee_residence VARCHAR(100),
    remote_ratio VARCHAR(50),
    required_skills VARCHAR(MAX), 
    education_required VARCHAR(50),
    years_experience INT,
    experience_range VARCHAR(20),  #new col
    industry VARCHAR(100),
    posting_date VARCHAR(20),
    benefits_score VARCHAR(20),
    company_name VARCHAR(255),
    posting_year INT  #new col
);

#populate the silver table with modifications#
TRUNCATE TABLE silver.ai_jobs_clean;

INSERT INTO silver.ai_jobs_clean (
    job_id, job_title, salary_usd, experience_level, employment_type,
    company_location, company_size, employee_residence, remote_ratio,
    required_skills, education_required, years_experience,experience_range, industry,
    posting_date,
    benefits_score, company_name, posting_year
)
SELECT
    job_id,
    TRIM(job_title),
    TRY_CAST(
    CASE 
        WHEN TRY_CAST(salary_usd AS DECIMAL(15,2)) <= 0 THEN NULL #check for outliers
        ELSE salary_usd
    END AS DECIMAL(15,2)
) AS salary_usd,
    CASE experience_level
        WHEN 'EN' THEN 'Entry'
        WHEN 'MI' THEN 'Mid'
        WHEN 'SE' THEN 'Senior'
        WHEN 'EX' THEN 'Executive'
        ELSE experience_level
    END AS experience_level,
    CASE employment_type
        when 'FL' THEN 'Freelance'
        WHEN 'PT' Then 'Part Time'
        WHEN 'CT' THEN 'Contract'
        WHEN 'FT' THEN ' Full Time'
        ELSE employment_type 
      END AS employment_type,
    company_location,
    CASE company_size
        WHEN 'S' THEN 'Small'
        WHEN 'M' THEN 'Medium'
        WHEN 'L' THEN 'Large'
        ELSE company_size
    END AS company_size,
    employee_residence,
    CASE remote_ratio
        WHEN '100' THEN 'Remote'
        WHEN '50' THEN 'Hybrid'
        WHEN '0' THEN 'On-site'
        ELSE remote_ratio
    END AS work_mode,
    required_skills,
    education_required,
    TRY_CAST(years_experience AS INT),
    CASE 
        WHEN years_experience  BETWEEN 0 AND 2 THEN '0–2'
        WHEN years_experience  BETWEEN 3 AND 5 THEN '3–5'
        WHEN years_experience  BETWEEN 6 AND 10 THEN '6–10'
        WHEN years_experience  > 10 THEN '10+'
        ELSE 'Unknown'
    END AS experience_range,
    industry,
    FORMAT(TRY_CAST(LTRIM(RTRIM(posting_date)) AS DATE), 'dd-MM-yyyy') AS posting_date, #formatting the date
    CASE 
        WHEN TRY_CAST(benefits_score AS DECIMAL(3,1)) IS NULL THEN 'Unknown'
        WHEN TRY_CAST(benefits_score AS DECIMAL(3,1)) < 6.0 THEN 'Poor'
        WHEN TRY_CAST(benefits_score AS DECIMAL(3,1)) < 7.5 THEN 'Fair'
        WHEN TRY_CAST(benefits_score AS DECIMAL(3,1)) < 8.5 THEN 'Good'
        WHEN TRY_CAST(benefits_score AS DECIMAL(3,1)) < 9.5 THEN 'Excellent'
        ELSE 'Outstanding'
    END AS benefits_category,
    company_name,
    YEAR(TRY_CAST(posting_date AS DATE)) AS posting_year # add the posting year col
FROM bronze.ai_jobs; 

#create table for job skills: # 
DROP TABLE IF EXISTS silver.job_skills;
CREATE TABLE silver.job_skills (
    job_id VARCHAR(50),
    skill_name VARCHAR(100)
);

INSERT INTO silver.job_skills (job_id, skill_name)
SELECT 
    job_id,
    TRIM(value) AS skill_name
FROM silver.ai_jobs_clean
CROSS APPLY STRING_SPLIT(required_skills, ',') 


# Silver Layer Validation

In [ ]:
#check for duplicates in the primary key(jod_id)
SELECT job_id, COUNT(*) AS count
FROM silver.ai_jobs_clean
GROUP BY job_id
HAVING COUNT(*) > 1; 

# text standarization (example for job_title variations) 
SELECT DISTINCT job_title
FROM silver.ai_jobs_clean
ORDER BY job_title;

#make sure all the data from bronze layer is imported
SELECT 
    (SELECT COUNT(*) FROM bronze.ai_jobs_raw) AS bronze_count,
    (SELECT COUNT(*) FROM silver.ai_jobs_clean) AS silver_count;

#check for reasonable numeric ranges
SELECT MIN(salary_usd) AS min_salary, MAX(salary_usd) AS max_salary
FROM silver.ai_jobs_clean;

#check the categorical values addded
SELECT DISTINCT benefits_score_category
FROM silver.ai_jobs_clean;

#Sample Data Review 
SELECT TOP 10 * FROM silver.ai_jobs_clean ORDER BY NEWID();

# review all dataset 
select * from silver.ai_jobs_clean;

# Gold Layer 
The Gold layer uses a star schema with one central fact table capturing AI job records and multiple descriptive dimensions (job, time, location, company, experience, work conditions, and skills). Surrogate keys are used to ensure efficient joins and optimal performance in BI tools.

In [ ]:
-#gOLD: date dimension : --> Job demand over time 
CREATE TABLE gold.dim_date (
    date_key INT IDENTITY(1,1) PRIMARY KEY,
    posting_date DATE,
    year INT,
    month INT,
    quarter VARCHAR(2)
);

INSERT INTO gold.dim_date (posting_date, year, month, quarter)
SELECT DISTINCT
    TRY_CONVERT(DATE, posting_date, 103) AS full_date,
    YEAR(TRY_CONVERT(DATE, posting_date, 103)) AS year,
    MONTH(TRY_CONVERT(DATE, posting_date, 103)) AS month,
    CONCAT('Q', DATEPART(QUARTER, TRY_CONVERT(DATE, posting_date, 103))) AS quarter
FROM silver.ai_jobs_clean
WHERE TRY_CONVERT(DATE, posting_date, 103) IS NOT NULL;

select * from gold.dim_date; 

#GOLD: job dimension : show roles and seniority -->Used for:Job demand by role and Avg salary by role -- 
CREATE TABLE gold.dim_job (
    job_key INT IDENTITY(1,1) PRIMARY KEY,
    job_title VARCHAR(255),
    experience_level VARCHAR(50)
);

INSERT INTO gold.dim_job
SELECT DISTINCT job_title, experience_level
FROM silver.ai_jobs_clean;

select * from gold.dim_job;

#GOLD: location dimension: geography: used for Job demand by country and salary variation -- 
CREATE TABLE gold.dim_location (
    location_key INT IDENTITY(1,1) PRIMARY KEY,
    company_location VARCHAR(100)
);

INSERT INTO gold.dim_location
SELECT DISTINCT company_location
FROM silver.ai_jobs_clean;
select * from gold.dim_location;


#GOLD: company dimansion :organization context -- 
CREATE TABLE gold.dim_company (
    company_key INT IDENTITY(1,1) PRIMARY KEY,
    company_name VARCHAR(255),
    company_size VARCHAR(50),
    industry VARCHAR(100)
);

INSERT INTO gold.dim_company
SELECT DISTINCT company_name, company_size, industry
FROM silver.ai_jobs_clean;
select * from gold.dim_company;

#gold: experiece dimensions: carrer profile-- 
CREATE TABLE gold.dim_experience (
    experience_key INT IDENTITY(1,1) PRIMARY KEY,
    years_experience INT,
    experience_range VARCHAR(20),
    education_required VARCHAR(50)
);

INSERT INTO gold.dim_experience
SELECT DISTINCT years_experience, experience_range, education_required
FROM silver.ai_jobs_clean;
select * from gold.dim_experience;

#GOLD: work dimensions: conditions -- 
CREATE TABLE gold.dim_work (
    work_key INT IDENTITY(1,1) PRIMARY KEY,
    employment_type VARCHAR(50),
    work_arrangement VARCHAR(50),
    benefits_score VARCHAR(50)
);

INSERT INTO gold.dim_work
SELECT DISTINCT employment_type, remote_ratio, benefits_score
FROM silver.ai_jobs_clean;
select * from gold.dim_work;

#GOLD: skill dimansions-- 
CREATE TABLE gold.dim_skill (
    skill_key INT IDENTITY(1,1) PRIMARY KEY,
    skill_name VARCHAR(100)
);

INSERT INTO gold.dim_skill
SELECT DISTINCT TRIM(value)
FROM silver.ai_jobs_clean
CROSS APPLY STRING_SPLIT(required_skills, ',');
select * from gold.dim_skill;

#FACT TABLE --
CREATE TABLE gold.fact_ai_jobs (
    fact_id INT IDENTITY(1,1) PRIMARY KEY,
    date_key INT,
    job_key INT,
    location_key INT,
    company_key INT,
    experience_key INT,
    work_key INT,
    salary_usd DECIMAL(15,2),
    job_count INT DEFAULT 1
);
INSERT INTO gold.fact_ai_jobs (
    date_key,
    job_key,
    location_key,
    company_key,
    experience_key,
    work_key,
    salary_usd,
    job_count
)
SELECT
    d.date_key,
    j.job_key,
    l.location_key,
    c.company_key,
    e.experience_key,
    w.work_key,
    s.salary_usd,
    1
FROM (
    SELECT *,
           TRY_CONVERT(DATE, posting_date, 103) AS posting_date_clean
    FROM silver.ai_jobs_clean
) s
LEFT JOIN gold.dim_date d
    ON s.posting_date_clean = d.posting_date
LEFT JOIN gold.dim_job j
    ON s.job_title = j.job_title
   AND s.experience_level = j.experience_level
LEFT JOIN gold.dim_location l
    ON s.company_location = l.company_location
LEFT JOIN gold.dim_company c
    ON s.company_name = c.company_name
LEFT JOIN gold.dim_experience e
    ON s.years_experience = e.years_experience
   AND s.education_required = e.education_required
LEFT JOIN gold.dim_work w
    ON s.remote_ratio = w.work_arrangement
   AND s.benefits_score = w.benefits_score
WHERE s.posting_date_clean IS NOT NULL;

select * from gold.fact_ai_jobs;



# Gold layer validation 


In [ ]:
#Check how many rows loaded:
SELECT COUNT(*) FROM gold.fact_ai_jobs;

#check missing dates
SELECT COUNT(*) 
FROM gold.fact_ai_jobs
WHERE date_key IS NULL;